In [61]:
import pandas as pd
import numpy as np

In [62]:
#normalization
def normalize_(X,mean,std):
    X_new = (X-mean)/std;
    return X_new

def normalize(X):
    mean = np.mean(X,axis=0);
    std = np.std(X,axis=0);
    std[std < 0.001] =1
    X_new = (X-mean)/std;
    return X_new, mean, std

def prepare_X(X):
    m = X.shape[0]
    ones = np.ones((m, 1))
    X_new = np.column_stack((ones, X))
    return X_new

def sigmoid(z): #where z is column vector
    g = 1/(1+np.exp(z*-1));
    return g

def h(X, theta): #where X is matrix and theta is vector(!)
    z = X.dot(theta)
    return sigmoid(z)

#cost function (штрафна функція)
def cost_function(X, y, theta):
    m = X.shape[0]
    if m ==0:
        return None
    temp =h(X,theta);
    J = sum(-1*y*np.log(temp) - (1-y)*np.log(1-temp) )/m;
    return J

def derivative_theta(X, y, theta):
    m = X.shape[0];
    if m == 0:
        return None
    d_theta = X_new.T.dot(h(X,theta) -y)/m;
    
    return d_theta

#gradient descent (градієнтний спуск)
def gradient_descent(X, y, theta, alpha, epsilon, num_iters, print_J = True):
    m = X.shape[0]
    J_history = [] 
    J = cost_function(X,y,theta);
    
    if print_J == True:
        print(J)
    J_history.append(J)
    for i in range(num_iters):
        theta -= alpha* derivative_theta(X,y,theta);
        J = cost_function(X,y,theta);
        
        J_history.append(J)
        
        if i % 1000 == 0 and print_J == True:
            print(J)
        
        if abs(J-J_history[-2]) < epsilon:
            break
            
    return theta, J_history

#prediction method
def predict(X, mean, std, models,labels):
    
    X_new =normalize_(X,mean,std);
    X_new = prepare_X(X_new);
    predictions = [];
    for theta in models:
        predictions.append(h(X_new,theta))
    
    predict =[];
    for p in range(0,len(predictions[0])):
        max = abs(predictions[0][p]);
        max_label  =labels[0];
        for l in range(1,len(labels)):
            if(abs(predictions[l][p]) > max):
                max = abs(predictions[l][p]);
                max_label  =labels[l];
        predict.append(max_label);
    return predict

In [64]:
dataset = pd.read_csv('train.csv');
dataset.describe() #0 - pixel is black

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [65]:
corr_matrix = dataset.corr() #correlation matrix
corr_matrix['label'].sort_values(ascending = False)

label       1.000000
pixel381    0.431506
pixel409    0.423767
pixel436    0.420922
pixel408    0.413855
              ...   
pixel760         NaN
pixel780         NaN
pixel781         NaN
pixel782         NaN
pixel783         NaN
Name: label, Length: 785, dtype: float64

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
X = dataset.drop(['label'], axis=1)
y = dataset['label']
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)


In [69]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.1, random_state=42)
X_train.shape
X_test.shape

(4200, 784)

In [70]:
clf = LogisticRegression(max_iter =100).fit(X_train, y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
clf.coef_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [54]:
predicted = clf.predict(X_test)

In [55]:
delta =(predicted == y_test)

In [56]:
unique, counts = np.unique(y_test[~delta], return_counts=True)
print("Failed to predict numbers:",dict(zip(unique, counts)))

Failed to predict numbers: {0: 16, 1: 15, 2: 56, 3: 70, 4: 38, 5: 48, 6: 29, 7: 38, 8: 52, 9: 44}


In [57]:
clf.score(X_train,y_train)

0.9530952380952381

In [58]:
clf.score(X_test,y_test)

0.9033333333333333

In [59]:
accuracy_score(y_test,predicted)

0.9033333333333333

In [74]:
X = dataset.drop(['label'], axis=1)
y = dataset['label']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.1, random_state=42)

In [77]:
from sklearn.preprocessing import OneHotEncoder
print('Training set: X={}, y={}'.format(X_train.shape, y_train.shape))
print('Test set: X={}, y={}'.format(X_test.shape, y_test.shape))
enc = OneHotEncoder(handle_unknown='error')
enc.fit(np.array(y).reshape(-1,1))
OneHotEncoder()
enc.categories_

Training set: X=(37800, 784), y=(37800,)
Test set: X=(4200, 784), y=(4200,)


[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)]

In [78]:
y_train_encoded  = enc.transform(np.array(y_train).reshape(-1,1)).toarray()
models =[]
X_new,mean,std = normalize(X_train)
X_new = prepare_X(X_new)

In [81]:
for m in [0,1,2,3,4,5,6,7,8,9]:
    theta = np.zeros((X_new.shape[1], 1))
    y_new =y_train_encoded[:,m].reshape((X_train.shape[0], 1))
    new_theta, Js = gradient_descent(X_new, y_new, theta, 0.01, 1e-7, 100, False)
    print("Cost for label ",m," is ",cost_function(X_new, y_new, theta))
    models.append(theta)

Cost for label  0  is  [0.41378976]
Cost for label  1  is  [0.43043425]
Cost for label  2  is  [0.44405079]
Cost for label  3  is  [0.46531496]
Cost for label  4  is  [0.47119298]
Cost for label  5  is  [0.4892763]
Cost for label  6  is  [0.42314857]
Cost for label  7  is  [0.42881488]
Cost for label  8  is  [0.48672177]
Cost for label  9  is  [0.49155582]


In [83]:
y_train_pred=predict(X_train,mean,std,models,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
accuracy_score(y_train, y_train_pred)

0.7686243386243387

In [84]:
y_test_pred =predict(X_test,mean,std,models,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
delta =((y_test == y_test_pred))
delta[delta == False]

5457     False
25536    False
31803    False
17164    False
34680    False
         ...  
33306    False
20250    False
3615     False
25795    False
35267    False
Name: label, Length: 955, dtype: bool

In [98]:
accuracy_score(y_test, y_test_pred)

0.7726190476190476